In [1]:
#!pip install geopy
#!pip install folium

# The webpage with the postal codes of Toronto Canada will be scrapped to extract the details in a Dataframe using beautiful soup
with algorithm 'lxml'. Relatively most efficient to extract by navigating the tree

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

from collections import Counter
import json
from sklearn.cluster import KMeans
from pandas import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

column_names = ['PostalCode', 'Borough', 'Neighborhood']
t_rows = 0
t_cols = 0
area = []
borough=[]
neighborhood=[]
postalcode = []

source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [3]:
cols = soup.find('td')

In [4]:
postalcode = cols.b.text
print(postalcode)

M1A


In [5]:
span = cols.span.text
print(span)

Not assigned


In [6]:
postalcode=[]
areadetails=[]

for cols in soup.find_all('td'):
    if cols.b != None:
        postalcode.append(cols.b.text)
    if cols.span != None:
        areadetails.append(cols.span.text)

In [7]:
for x in areadetails:
    boroname = ""
    neighdetails = ""
    if (x.find("(") != -1):
        boroname = x[:x.find("(")]
        neighdetails = x[x.find("("):]
        neighdetails = neighdetails.replace("(","")
        neighdetails = neighdetails.replace(")","")
        neighdetails = neighdetails.replace(" / ", ", ")
        borough.append(boroname)
        neighborhood.append(neighdetails)
    else:
        borough.append('Not assigned')
        neighborhood.append('Not assigned')

In [8]:
df = pd.DataFrame([postalcode, borough, neighborhood])
df_Trans = df.T
df_Trans.head(5)

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
df_Trans.columns = ['Postalcode', 'Borough', 'Neighborhood']

In [10]:
df_Trans.head(5)
df_Trans.drop(df_Trans[df_Trans['Borough']=='Not assigned'].index, inplace=True)

# The dataframe is created with 3 columns, Postalcode, Borough & Neighborhood

In [11]:
toronto_df = df_Trans.reset_index(drop=True)
toronto_df.head(5)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [12]:
toronto_df.shape

(103, 3)

# Using the geospatial dataset, the latitude and longitude coordinates has been extracted from the file for the postal codes 

In [13]:
geo_coordinate_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_coordinate_df.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
req_pcode = toronto_df['Postalcode'].to_list()

In [15]:
geo_req_df = geo_coordinate_df[geo_coordinate_df['Postal Code'].isin (req_pcode)]

In [16]:
geo_req_df.sort_values(by='Postal Code', ascending=True)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [17]:
toronto_df_sorted = toronto_df.sort_values(by='Postalcode', ascending=True)

In [18]:
toronto_df_sorted = toronto_df_sorted.set_index('Postalcode').join(geo_req_df.set_index('Postal Code'))

Combined dataframe with postalcode, borough, neighborhood, latitude and longitude has been generated.

In [19]:
toronto_df_sorted.head(5)

,Borough,Neighborhood,Latitude,Longitude
Postalcode,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
